# Import Model

In [1]:
import torch
import torch.nn as nn


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, negative_slope=0.01):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.LeakyReLU(negative_slope)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(out_channels)

        # 如果 in_channels != out_channels 或 stride != 1，就需要用 1×1 conv 來對齊維度/尺寸
        self.shortcut = None
        if (stride != 1) or (in_channels != out_channels):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        
        self.relu2 = nn.LeakyReLU(negative_slope)

    def forward(self, x):
        identity = x  # 先把輸入保留下來

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # 如果需要調整通道或stride，就用shortcut
        if self.shortcut is not None:
            identity = self.shortcut(x)

        # 將殘差 (out) 與 identity 相加
        out += identity  
        out = self.relu2(out)
        return out
    
class ResNetLikeEncoder(nn.Module):
    def __init__(self, out_dim, in_channels=3, negative_slope=0.01):
        super().__init__()
        # 第一段：輸入後馬上下採樣
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(negative_slope),
            nn.MaxPool2d(kernel_size=2)
        )
        # 第二段：殘差塊 x2，調通道 16 → 32
        self.layer1 = nn.Sequential(
            ResidualBlock(16, 32, stride=1, negative_slope=negative_slope),
            ResidualBlock(32, 32, stride=1, negative_slope=negative_slope),
            nn.MaxPool2d(kernel_size=2)
        )
        # 第三段：殘差塊 x2，調通道 32 → 64
        self.layer2 = nn.Sequential(
            ResidualBlock(32, 64, stride=1, negative_slope=negative_slope),
            ResidualBlock(64, 64, stride=1, negative_slope=negative_slope),
            nn.AdaptiveAvgPool2d((1, 1))  # 池化到 1×1
        )
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(64, out_dim)

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x


class MLPDecoder(nn.Module):
    def __init__(self, input_dim, output_dim=35, negative_slope=0.01):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(p=0.1),  # 調整 dropout 至 0.1
            nn.Linear(128, 64),
            nn.LeakyReLU(negative_slope),
            nn.Dropout(p=0.1),   # 調整 dropout 至 0.1
            nn.Linear(64, output_dim)  # 👉 Linear activation
        )
        # 每個 head 對應一個 cell type 的輸出

    def forward(self, x):
        outs = self.shared(x)
        # 將所有 head 的輸出合併成 (B, output_dim)
        return outs



class VisionMLP_MultiTask(nn.Module):
    def __init__(self, cnn_out_dim=64, output_dim=35, negative_slope=0.01):
        super().__init__()
        self.encoder_tile = ResNetLikeEncoder(cnn_out_dim, in_channels=3, negative_slope=negative_slope)
        self.encoder_neighbors = ResNetLikeEncoder(cnn_out_dim, in_channels=3, negative_slope=negative_slope)
        self.encoder_subtiles = ResNetLikeEncoder(cnn_out_dim, in_channels=3, negative_slope=negative_slope)

        # 注意這裡更新 decoder 的 input_dim，如果不需要坐標可以另外調整
        self.decoder = MLPDecoder(input_dim=cnn_out_dim * 3 + 2,
                                  output_dim=output_dim,
                                  negative_slope=negative_slope)

    def forward(self, tile, subtiles, neighbors, coords):
        B = tile.size(0)

        # tile: [B, 3, 78, 78]
        f_tile = self.encoder_tile(tile)

        # subtiles: [B, 9, 3, 26, 26]
        B, N, C, H, W = subtiles.shape
        subtiles_reshaped = subtiles.contiguous().reshape(B * N, C, H, W)
        f_sub = self.encoder_subtiles(subtiles_reshaped).reshape(B, N, -1).mean(dim=1)

        # neighbors: [B, 8, 3, 78, 78]
        B, N, C, H, W = neighbors.shape
        neighbors_reshaped = neighbors.contiguous().reshape(B * N, C, H, W)
        f_neigh = self.encoder_neighbors(neighbors_reshaped).reshape(B, N, -1).mean(dim=1)

        x = torch.cat([f_tile, f_sub, f_neigh, coords], dim=1)
        return self.decoder(x)






model = VisionMLP_MultiTask()

## Load Model

In [ ]:

from python_scripts.import_model import load_model_classes
from python_scripts.operate_model import get_model_inputs

# ==============================================
# 範例使用
# ==============================================
folder = "./output_folder/CNN+MLP/"  # 替換成實際的資料夾路徑，該路徑下應有 model.py
try:
    loaded_classes = load_model_classes(folder)
    print("載入的 class 名稱:", list(loaded_classes.keys()))
except Exception as e:
    print("載入模型 class 發生錯誤:", e)    


載入的 class 名稱: ['CNNEncoder', 'MLPDecoder', 'VisionMLPModelWithCoord']


In [7]:
# 假設 loaded_classes 是你已經從 model.py 載入的 class 字典
name_of_class = 'VisionMLPModelWithCoord'
ModelClass = loaded_classes.get(name_of_class)
if ModelClass is None:
    raise ValueError(f"找不到 {name_of_class} 這個 class")
# 這裡呼叫 ModelClass() 建立實例，注意不能直接用 name_of_class() (因為它是一個字串)
model = ModelClass()  # 如果需要參數，請在此處傳入
get_model_inputs(model)
#print("建立的模型實例:", model)


Model forward signature: (center_tile, subtiles, neighbor_tiles, coords)


<Signature (center_tile, subtiles, neighbor_tiles, coords)>

# Import training data

## Same in multiple .pt

In [2]:
import os
import torch

def load_grouped_tile_data(folder_path):
    neighbors_list = []
    tiles_list = []
    subtiles_list = []
    labels_list = []
    #coords_list = []
    norm_coords_list = []

    pt_files = sorted([
        f for f in os.listdir(folder_path)
        if f.endswith(".pt") and not f.startswith("original")
    ])

    for fname in pt_files:
        file_path = os.path.join(folder_path, fname)
        try:
            data = torch.load(file_path, map_location="cpu")
        except Exception as e:
            print(f"❌ Failed to load {fname}: {e}")
            continue  # skip this file
        
        x, y = data["norm_coord"]
        standard_coor = torch.tensor([x, y]) * 2 - 1  # 這樣是 shape = [2]

        neighbors_list.append(data["neighbors"])
        tiles_list.append(data["tile"])
        subtiles_list.append(data["subtiles"])
        labels_list.append(data["label"])
        #coords_list.append(data["coord"])
        norm_coords_list.append(standard_coor)
    
    return {
        "neighbors": neighbors_list,
        "tile": tiles_list,
        "subtiles": subtiles_list,
        "label": labels_list,
        "coords": norm_coords_list,
        #"norm_coord": norm_coords_list
    }

# 🧪 用法
grouped_data = load_grouped_tile_data("./dataset/3-channel/masked-78-3sizes-3*3_log2-gu-zscore/train")


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_33678/2562856491.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path, map_location="c

In [3]:
from python_scripts.import_data import importDataset

image_keys = ['neighbors', 'tile', 'subtiles']

train_dataset = importDataset(
        data_dict=grouped_data,
        model=model,
        image_keys=image_keys,
        transform=lambda x: x,  # identity transform
        print_sig=True
    )

/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon

Model forward signature: (tile, subtiles, neighbors, coords)


In [4]:
train_dataset.check_item(1000, 10)


🔍 Checking dataset sample: 1000
📏 tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: 0.000, max: 0.960, mean: 0.328, std: 0.370
📏 subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32 | min: 0.000, max: 0.960, mean: 0.328, std: 0.370
📏 neighbors shape: torch.Size([8, 3, 78, 78]) | dtype: torch.float32 | min: 0.000, max: 0.998, mean: 0.334, std: 0.368
📏 coords shape: torch.Size([2]) | dtype: torch.float32 | min: 0.610, max: 0.850, mean: 0.730, std: 0.170
--- coords head (前 10 個元素):
tensor([0.6097, 0.8500])
📏 label shape: torch.Size([35]) | dtype: torch.float32 | min: -1.127, max: 2.526, mean: -0.344, std: 0.707
--- label head (前 10 個元素):
tensor([-0.7291,  0.0241, -0.6995, -0.6637,  2.5256, -0.7112,  0.5876, -0.4779,
        -0.6379, -0.3177])
✅ All checks passed!


In [5]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader

# 設定比例
train_ratio = 0.8
val_ratio = 1 - train_ratio
total_len = len(train_dataset)
train_len = int(train_ratio * total_len)
val_len = total_len - train_len

# 拆分 Dataset
generator = torch.Generator().manual_seed(42)
train_set, val_set = random_split(train_dataset, [train_len, val_len], generator=generator)

print(f"✅ Train: {len(train_set)} samples")
print(f"✅ Val: {len(val_set)} samples")

# 🔹 將其包成 DataLoader
BATCH_SIZE = 32

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)


✅ Train: 26179 samples
✅ Val: 6545 samples


# Save in one pt

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
from python_scripts.import_data import importDataset, preprocess_data
from torch.utils.data import DataLoader
from torchvision import transforms

# 載入資料
#tile = torch.load("./dataset/final_data/M_tiles.pt")
#subtiles = torch.load("./train_dataset_sep_v2/subtiles.pt")
#neighbor_tiles = torch.load("./train_dataset_sep_v2/neighbor_tiles.pt")
#label = torch.load("./dataset/final_data/gu_log2_labels.pt")
#meta = torch.load("./dataset/final_data/meta_info.pt")



raw_coords = []
for _meta in grouped_data['coords']:
    if _meta is not None:
         x, y = _meta
    else:
        x, y = 0, 0
    raw_coords.append([x, y])

raw_coords = np.array(raw_coords)


coord_scaler = StandardScaler()
normalized_coords = coord_scaler.fit_transform(raw_coords)

train_dataset = {
        'tile': tile,
        #'subtiles': subtiles,
        #'neighbor_tiles': neighbor_tiles,
        'coords': normalized_coords,
        'label': label
    }

my_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])

image_keys = ['tile']

#processed_data = preprocess_data(train_dataset, image_keys, my_transform)

train_dataset = importDataset(
        data_dict=train_dataset,
        model=model,
        image_keys=image_keys,
        transform=lambda x: x,  # identity transform
        print_sig=True
    )

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_19297/1962845458.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tile = torch.load("./dataset/final_data/M_til

Model forward signature: (tile, coords)


In [3]:
train_dataset = importDataset(
        data_dict=processed_data,
        model=model,
        image_keys=image_keys,
        transform=lambda x: x,  # identity transform
        print_sig=True
        )

NameError: name 'processed_data' is not defined

In [5]:
train_dataset.check_item(1000, 10)


ValueError: 轉換欄位 'tile' 的資料為 tensor 時出錯，資料內容: [[[0.9678768  0.954182   0.9747549 ]
  [0.9704657  0.95502454 0.9764706 ]
  [0.9704657  0.95502454 0.9764706 ]
  ...
  [0.9767846  0.95091146 0.9703432 ]
  [0.9713082  0.9556219  0.96738666]
  [0.9646906  0.95493263 0.9654565 ]]

 [[0.9689032  0.95465684 0.97550553]
  [0.97071075 0.95477945 0.9764706 ]
  [0.96967673 0.9537454  0.9754366 ]
  ...
  [0.97511876 0.9502413  0.97131205]
  [0.9694317  0.95374537 0.96551013]
  [0.9623774  0.9550245  0.9647059 ]]

 [[0.9686274  0.95686275 0.9764706 ]
  [0.9676509  0.9558862  0.975494  ]
  [0.96568245 0.95391774 0.97352564]
  ...
  [0.9730354  0.9502413  0.9730354 ]
  [0.9664216  0.9529412  0.9647059 ]
  [0.96134347 0.9550245  0.9647059 ]]

 ...

 [[0.8678539  0.6751915  0.78875613]
  [0.89483    0.7065947  0.8041437 ]
  [0.91975725 0.7323836  0.8442517 ]
  ...
  [0.68689877 0.554902   0.77635956]
  [0.68630135 0.5452283  0.7674058 ]
  [0.7217984  0.57002914 0.78608304]]

 [[0.8391927  0.6464614  0.7

In [23]:
from torch.utils.data import random_split

# 設定比例
train_ratio = 0.8
val_ratio = 1 - train_ratio
total_len = len(train_dataset)
train_len = int(train_ratio * total_len)
val_len = total_len - train_len

# 拆分 Dataset
generator = torch.Generator().manual_seed(42)
train_set, val_set = random_split(train_dataset, [train_len, val_len], generator=generator)

print(f"✅ Train: {len(train_set)} samples")
print(f"✅ Val: {len(val_set)} samples")

# 🔹 將其包成 DataLoader
BATCH_SIZE = 32

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)


✅ Train: 26179 samples
✅ Val: 6545 samples


#  Train model

In [6]:
import os
import torch
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from python_scripts.operate_model import train_one_epoch, evaluate, predict, EarlyStopping, plot_losses, plot_per_cell_metrics
import torch.nn as nn

# ---------------------------
# 指定儲存資料夾
# ---------------------------
save_folder = "output_folder/with_all_preprocess_log2/Masked-CNN+Res+MLP_3size_78_log2-gu-zscore/"  # 修改為你想要的資料夾名稱
if not os.path.exists(save_folder):
    os.makedirs(save_folder)


In [7]:
# ---------------------------
# 🔧 設定裝置
# ---------------------------
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ---------------------------
# 初始化模型 & 優化器
# ---------------------------
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
#loss_fn = lambda pred, tgt: hybrid_weighted_spearman_loss(pred, tgt, alpha=0.5, beta=1.0)

#loss_fn = nn.MSELoss()
# loss_fn = spearman_loss
#LOSS_TYPE = "weighted_mse"  # 或 "mse", "mae", "spearman"


#loss_fn = get_loss_fn(loss_type=LOSS_TYPE, cell_weights=cell_weights)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
early_stopper = EarlyStopping(patience=10)

# ---------------------------
# 儲存 log 的設定
# ---------------------------
log_path = os.path.join(save_folder, "training_log.csv")
log_file = open(log_path, mode="w", newline="")
csv_writer = csv.writer(log_file)
csv_writer.writerow(["Epoch", "Train Loss", "Val Loss", "Val Spearman", "Learning Rate"])

# ---------------------------
# 用來畫圖的變數
# ---------------------------
train_losses = []
val_losses = []
train_spearmanes = []
val_spearmanes = []

# ---------------------------
# 指定最佳模型儲存路徑
# ---------------------------
best_model_path = os.path.join(save_folder, "best_model.pt")
loss_plot_path = os.path.join(save_folder, "loss_curve.png")

# ---------------------------
# 開始訓練
# ---------------------------
num_epochs = 500
best_val_loss = float('inf')
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
display(fig)  # 初始顯示圖形


for epoch in range(num_epochs):
    train_loss, train_spearman = train_one_epoch(model, train_loader, optimizer, device,current_epoch= epoch,target_epoch =20 )
    val_loss, val_spearman, mse_per_cell, spearman_per_cell = evaluate(model, val_loader, device,current_epoch= epoch,target_epoch =20)
    clear_output(wait=True)  # 清除之前的輸出
    axes[0][0].clear()
    axes[0][1].clear()
    axes[1][0].clear()
    axes[1][1].clear()
    # 儲存最佳模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        print("✅ Saved best model!")
        

    # 調整學習率
    scheduler.step(val_loss)

    # 寫入 CSV log
    lr = optimizer.param_groups[0]['lr']
    csv_writer.writerow([epoch + 1, train_loss, val_loss, val_spearman, lr])

    # 印出 Epoch 結果
    print(f"Epoch {epoch + 1}/{num_epochs} | loss: {train_loss:.4f} | val_loss: {val_loss:.4f} | lr: {lr:.2e}")
    print(f"train spearman: {train_spearman:.4f} | Val spearman: {val_spearman:.4f} | ρ: {val_spearman:.4f}")

    # 更新 log 列表並畫圖
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_spearmanes.append(train_spearman)
    val_spearmanes.append(val_spearman)
    


    # plot loss
    plot_losses(train_losses, val_losses, ax=axes[0][0], title="MSE Loss")
    plot_losses(train_spearmanes, val_spearmanes, ax=axes[0][1], title="Spearman Loss")

    cell_names = [f"C{i+1}" for i in range(35)]
    # plot per-cell stats
    plot_per_cell_metrics(mse_per_cell, spearman_per_cell,cell_names,
                        ax_mse=axes[1][0], ax_spearman=axes[1][1])
    

    plt.tight_layout()
    plt.show()
    display(fig)
    plt.pause(0.1)  # 暫停以便更新畫面
    fig.savefig(loss_plot_path)
    print(f"曲線圖已儲存至 {loss_plot_path}")

    # Early stopping
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("⛔ Early stopping triggered.")
        break

# 關閉 CSV log 檔案
log_file.close()

# 儲存最終圖形
plt.close(fig)
print(f"訓練結束，loss 曲線圖已儲存至 {loss_plot_path}")


Epoch 19/500 | loss: 0.1831 | val_loss: 0.1834 | lr: 1.00e-04
train spearman: 0.8447 | Val spearman: 0.8676 | ρ: 0.8676


<Figure size 640x480 with 0 Axes>

曲線圖已儲存至 output_folder/with_all_preprocess_log2/Masked-CNN+Res+MLP_3size_78_log2-gu-zscore/loss_curve.png
EarlyStopping counter: 3/10


KeyboardInterrupt: 

# Integrated Gradient

In [9]:
val_set[0]

{'tile': tensor([[[0.7668, 0.7243, 0.7168,  ..., 0.8300, 0.8208, 0.7723],
          [0.7256, 0.6888, 0.7023,  ..., 0.7706, 0.7576, 0.6791],
          [0.7218, 0.7186, 0.7368,  ..., 0.7060, 0.6706, 0.6279],
          ...,
          [0.7892, 0.7831, 0.8269,  ..., 0.7174, 0.6990, 0.6717],
          [0.7236, 0.7350, 0.7941,  ..., 0.7189, 0.6729, 0.6972],
          [0.7661, 0.7600, 0.7574,  ..., 0.7694, 0.7536, 0.7565]],
 
         [[0.6600, 0.6145, 0.6064,  ..., 0.6508, 0.6443, 0.5940],
          [0.6180, 0.5790, 0.5873,  ..., 0.5941, 0.5810, 0.4993],
          [0.6119, 0.6078, 0.6185,  ..., 0.5295, 0.4916, 0.4424],
          ...,
          [0.6837, 0.6650, 0.7063,  ..., 0.5808, 0.5657, 0.5393],
          [0.6053, 0.6135, 0.6569,  ..., 0.5907, 0.5408, 0.5730],
          [0.6407, 0.6252, 0.6123,  ..., 0.6439, 0.6278, 0.6311]],
 
         [[0.8415, 0.8027, 0.7979,  ..., 0.8703, 0.8713, 0.8255],
          [0.7992, 0.7672, 0.7704,  ..., 0.8196, 0.8130, 0.7472],
          [0.7843, 0.7789, 0.794

In [8]:
from python_scripts.operate_model import automate_ig_analysis

image_dict = {
    'tile': True,
    'subtiles': True,
    'neighbors': True,
    'coords': False
}

# 使用示例：從 DataLoader 取出一個 batch，然後呼叫：
for batch in val_loader:
    attr_results = automate_ig_analysis(model, batch, target_index=20, n_steps=50, image_dict=image_dict)
    break


Model forward signature: (tile, subtiles, neighbors, coords)
Model input parameters: ['tile', 'subtiles', 'neighbors', 'coords']


RuntimeError: Mismatched Tensor types in NNPack convolutionOutput

# Predict

In [10]:
# ===== 需要的 Libraries =====
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import csv
import os

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")
best_model_path = os.path.join(save_folder, "best_model.pt")
model = model.to(device)

model.load_state_dict(torch.load(best_model_path))
model.eval()


✅ Using device: mps


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_33678/3109279275.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_

VisionMLP_MultiTask(
  (encoder_tile): ResNetLikeEncoder(
    (layer0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): LeakyReLU(negative_slope=0.01)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine

In [ ]:
raw_coords = []
for _meta in test_data['coords']:
    if _meta is not None:
        x, y = _meta
    else:
        x, y = 0, 0
    raw_coords.append([x, y])

raw_coords = np.array(raw_coords)

coord_scaler = StandardScaler()
normalized_coords = coord_scaler.fit_transform(raw_coords)


In [ ]:
import torch
from torch.utils.data import Dataset
import inspect
from python_scripts.import_data import importDataset, preprocess_data
from python_scripts.operate_model import get_model_inputs
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# 正確方式

test_data = torch.load("./dataset/3-channel/masked-78-3sizes-3*3_log2-gu-zscore/test/test_dataset.pt")

raw_coords = []
for _meta in test_data['normal_coords']:
    x, y = _meta
    standard_coor = torch.tensor([x, y]) * 2 - 1  # 這樣是 shape = [2]

    raw_coords.append(standard_coor)

test_dataset = {
        'tile':  test_data['tiles'],
        'subtiles': test_data['subtiles'],
        'neighbors':  test_data['neighbor_tiles'],
        'coords': raw_coords,
        'label': np.zeros((len(test_data['tiles']), 35))
    }


image_keys = ['tile','subtiles', 'neighbors']



#processed_data = preprocess_data(test_dataset, image_keys, my_transform)

test_dataset = importDataset(
        data_dict=test_dataset,
        model=model,
        image_keys=image_keys,
        transform=lambda x: x,  # identity transform
        print_sig=True
    )


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_33678/2499438941.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load("./dataset/3-channel/

In [ ]:

test_dataset.check_item(1000, 10)


🔍 Checking dataset sample: 1000
📏 tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: 0.200, max: 1.000, mean: 0.729, std: 0.152
📏 subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32 | min: 0.200, max: 1.000, mean: 0.729, std: 0.152
📏 neighbors shape: torch.Size([8, 3, 78, 78]) | dtype: torch.float32 | min: 0.153, max: 1.000, mean: 0.732, std: 0.134
📏 coords shape: torch.Size([2]) | dtype: torch.float32 | min: -0.112, max: 0.166, mean: 0.027, std: 0.197
--- coords head (前 10 個元素):
tensor([-0.1120,  0.1663])
📏 label shape: torch.Size([35]) | dtype: torch.float32 | min: 0.000, max: 0.000, mean: 0.000, std: 0.000
--- label head (前 10 個元素):
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
✅ All checks passed!


In [15]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [31]:
test_preds = predict(model, test_loader, device)
test_preds

array([[-0.704486  ,  1.5522823 , -0.67788583, ...,  2.1145058 ,
        -0.0356152 ,  0.03656982],
       [-0.37445247,  1.5341558 , -0.34108874, ...,  1.7411622 ,
         0.3115291 ,  0.38824564],
       [ 0.17455263,  0.49443793,  0.2711917 , ...,  0.89854157,
         0.8972608 ,  0.61311704],
       ...,
       [ 0.21023479, -0.54313445,  0.220445  , ..., -0.43817687,
        -0.46568   , -0.6096501 ],
       [ 0.23745254, -0.40017462,  0.2594197 , ...,  0.03448338,
        -0.09314237, -0.44221896],
       [-0.22471237,  0.61608034, -0.13668048, ...,  0.92857766,
         0.35924202,  0.30561155]], dtype=float32)

In [ ]:
from python_scripts.revert_utils import (
    load_json_params,
    revert_log2_predictions,
    revert_boxcox_predictions
)

# 還原 Box-Cox
params = load_json_params("./dataset/boxcox_zscore_params.json")
restored = revert_boxcox_predictions(test_preds, params)


🔁 測試 revert_log2_predictions
[[ 1.01396717e+00 -8.07002121e-03  1.80511435e+00  1.13151159e-01
   2.00707840e+00  7.59623606e-02  6.96093327e-02  7.63244836e-03
  -4.43450703e-02  4.99186394e-04  1.08874982e-02 -2.43959104e-02
   4.05517143e-03  2.29893147e-02  6.26831059e-02  8.12215671e-02
   2.64344670e-01  2.31829931e-02  1.36808254e-02 -1.33865438e-03
  -2.55863969e-02  1.04127442e-02  1.75559047e-03 -3.42817691e-02
   8.95906936e-02  2.71020152e-03  7.19174871e-02  1.14335168e-02
  -2.35451263e-02  5.75979741e-03  1.10113500e-01  8.37440865e-02
   1.95191196e-02  1.52221790e-02 -7.13266282e-02]
 [ 3.28478151e+00  3.26228103e-01 -1.51059413e-01  2.10342036e-01
  -3.70963169e-01  9.46231454e-02  1.08288662e-02  4.10010213e-03
   3.99134882e-01  1.60557299e-02 -6.11376033e-03  1.01965609e-03
   7.99558069e-03  1.75983651e-02  1.73605903e-01  1.38104129e+00
   1.07006572e-02 -5.08118852e-02  1.18068944e-03  6.97391458e-02
  -2.20179089e-03  2.30087185e-02 -5.02672957e-03  1.61468807e

In [34]:
from python_scripts.revert_utils import (
    load_json_params,
    revert_log2_predictions,
)
#params = load_json_params(json_path="./dataset/spots-data/version-1/zscore_params.json")

# 或還原 log2
params = load_json_params("./dataset/spots-data/version-4/zscore_params.json")
restored = revert_log2_predictions(test_preds, params, add_constant=1)

restored

array([[0.03232527, 0.23497128, 0.04286408, ..., 0.02995825, 0.01395154,
        0.05367708],
       [0.20444083, 0.23338473, 0.16634631, ..., 0.02651131, 0.01888096,
        0.06989574],
       [0.5566361 , 0.14570951, 0.42948496, ..., 0.01877415, 0.02725267,
        0.08039701],
       ...,
       [0.5828054 , 0.06442869, 0.40558422, ..., 0.00661933, 0.00787783,
        0.02451277],
       [0.6030624 , 0.07527566, 0.42390466, ..., 0.01090062, 0.01313698,
        0.03199065],
       [0.29172492, 0.15563631, 0.24831343, ..., 0.01904893, 0.01956034,
        0.06606245]], dtype=float32)

In [35]:
import h5py

# ==== 讀取 test spot index 用於對應 ID ====
with h5py.File("./dataset/elucidata_ai_challenge_data.h5", "r") as f:
    test_spots = f["spots/Test"]
    test_spot_table = pd.DataFrame(np.array(test_spots['S_7']))  # Example: S_7

submission_path = os.path.join(save_folder, "submission.csv")

ensemble_df = pd.DataFrame(restored, columns=[f"C{i+1}" for i in range(restored.shape[1])])
ensemble_df.insert(0, 'ID', test_spot_table.index)
ensemble_df.to_csv(submission_path, index=False)
print(f"✅ Saved submission.csv in {submission_path}")


✅ Saved submission.csv in output_folder/with_all_preprocess_log2/CNN+Res+MTD_3size_78_log2-gu-zscore/submission.csv
